In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd


In [3]:
credits=pd.read_csv('tmdb_5000_credits.csv')
movies=pd.read_csv('tmdb_5000_movies.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'tmdb_5000_credits.csv'

In [ ]:
movies.head(1)


In [ ]:
credits.head(1)

In [ ]:
movies=movies.merge(credits,on='title')

In [ ]:
movies.head(1)

In [ ]:
movies=movies[['id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head(1)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies['genres'][1]

In [ ]:
import ast
def convert(obj):
    l=[];
    for i in ast.literal_eval(obj):
        l.append(i["name"])
    return l
        

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
print(movies['genres'])

In [ ]:
movies.keywords

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies.head(1).keywords

## movies.head(1)

In [ ]:
movies.iloc[0].crew

In [ ]:
import ast
def convert3(obj):
    l=[]
    n=1
    for i in ast.literal_eval(obj):
        if n<=3:
          l.append(i["name"])
          n=n+1
        else:
          break
    return l
        

In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
def fetch(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if(i['job']=='Director'):
          l.append(i['name'])
          break
    return l


In [ ]:
movies['crew']=movies['crew'].apply(fetch)


In [ ]:
movies.head(1)

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(1)

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['genres'][0]

In [ ]:
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])


In [ ]:
movies.head(1)

In [ ]:
movies['tag']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
new_df=movies[['id','title','tag']]

In [ ]:
print(new_df)

In [ ]:
new_df['tag']=new_df['tag'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tag']=new_df['tag'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
from nltk.stem.porter  import PorterStemmer 
ps=PorterStemmer()

In [ ]:
def root(text):
    l=[]
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)
                 
    

In [ ]:
new_df.shape

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(new_df['tag']).toarray()

In [ ]:
vectors

In [ ]:
cv.get_feature_names_out()

In [ ]:
new_df['tag']=new_df['tag'].apply(root)

In [ ]:
vectors

In [ ]:
cv.get_feature_names_out()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)
similarity

In [ ]:
ex=enumerate(similarity[0])
sorted(list(ex),reverse=True,key=lambda x: x[1])

In [ ]:
new_df[new_df['title']=='Avatar'].index[0]

In [ ]:
similarity[0]

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[0:5]

In [ ]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[0:5]
    for i in movies_list:
        print(movies.iloc[i[0]].title) 
    

In [ ]:
recommend('Avatar')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))